In [1]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using HDF5
using DynamicPolynomials
using Plots
using Dates

using Optim
using Random

Consider simple Lindblad master equation with two dissipators:

$
    \frac{d\rho}{dt} = -\frac{i}{\hbar} [H,\rho(t)] + \mathcal{D}[\rho(t)] = - \frac{i}{\hbar}[H, \rho]+\sum_{\ell=1}^{s-1}\left[J_\ell \rho J_\ell^\dagger - \frac{1}{2}\left\{ J_\ell^\dagger J_\ell, \rho \right\} \right],
$

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} w & 0 \\ 0 & 0
   \end{pmatrix}
$

 withot loss of generality we can possibly look for jump operator of the form:

$
J_1 = \begin{pmatrix} 0 & a \\ 0 & 0 
   \end{pmatrix} = 
   \begin{pmatrix} 0 & \sqrt{\gamma_1} \\ 0 & 0 
   \end{pmatrix}
$

$
J_2 = \begin{pmatrix} d & 0 \\ 0 & -d 
   \end{pmatrix} = 
   \begin{pmatrix} \sqrt{\gamma_2} & 0 \\ 0 & -\sqrt{\gamma_2} 
   \end{pmatrix}
$

$
J_3 = \begin{pmatrix} 0 & 0 \\ e & 0 
   \end{pmatrix} = 
   \begin{pmatrix} 0 &  0 \\ \sqrt{\gamma_3} & 0 
   \end{pmatrix}
$

$
J_4 = \begin{pmatrix} 0 & p \\ p & 0 
   \end{pmatrix} = \sqrt{\gamma_4/3} 
   \begin{pmatrix} 0 & 1\\ 1 & 0 
   \end{pmatrix}
$

$
J_5 = \begin{pmatrix} 0 & -i p \\ i p & 0 
   \end{pmatrix} = \sqrt{\gamma_4/3}
   \begin{pmatrix} 0 & -i \\ i & 0 
   \end{pmatrix}
$

$
J_6 = \begin{pmatrix} p & 0 \\ 0 & -p
   \end{pmatrix} = \sqrt{\gamma_4/3}
   \begin{pmatrix} 1 & 0 \\ 0 & -1 
   \end{pmatrix}
$

In [36]:
# Define polynomial variables
@polyvar w α β #r ϕ e p
@polyvar r ϕ e p g

Hˢʸᵐᵇ = [ w   0
          0   0. ]

#Hˢʸᵐᵇ = [ w           α + im* β
#          α + im* β   0.        ]


J₁ˢʸᵐᵇ = [ 0    r   
           0    0 ]

J₂ˢʸᵐᵇ = [ 0       0
           r-g+e   0 ]

@polyvar a3[1:4]
@polyvar b3[1:4]

J₃ˢʸᵐᵇ = [ a3[1]+im*b3[1]    a3[2]+im*b3[2]
           a3[3]+im*b3[3]   -a3[1]-im*b3[1] ]

@polyvar a4[1:4]
@polyvar b4[1:4]
      
J₄ˢʸᵐᵇ = [ 0                a4[2]+im*b4[2]
           a4[3]+im*b4[3]   0               ]

#J₃ˢʸᵐᵇ = ϕ * [ 1    0
#               0   -1. ]

#J₃ˢʸᵐᵇ = [ 0   0
#           e   0. ]

#J₄ˢʸᵐᵇ = p * [ 0    1
#               1    0. ]

J₅ˢʸᵐᵇ = p * [ 0   -im
               im   0  ]

J₆ˢʸᵐᵇ = p * [ 1    0
               0   -1. ]


Jˢʸᵐᵇ = [J₁ˢʸᵐᵇ, J₂ˢʸᵐᵇ, J₃ˢʸᵐᵇ, J₄ˢʸᵐᵇ, J₅ˢʸᵐᵇ, J₆ˢʸᵐᵇ]

function g_objective(γᵢ, op_num)

    objₑₓ = 0

    for df_trn in train_files # loop over initial states

        #ρᵗʳⁿ, tᵗʳⁿ = LiPoSID.get_rho_series(data_dir*df_trn*"_2CUT_data.h5", γᵢ)
        ρᵗʳⁿ, tᵗʳⁿ = LiPoSID.get_rho_series(data_dir*df_trn*"_data.h5", γᵢ)
        
        if length(tᵗʳⁿ) > 1200 end_train = 1200 else end_train = length(tᵗʳⁿ) end
            
        ρᵗʳⁿ = convert(Vector{Matrix{ComplexF64}}, ρᵗʳⁿ[1:end_train])
        
        tᵗʳⁿ = convert(Vector{Float64}, tᵗʳⁿ[1:end_train])

        objₑₓ += LiPoSID.simpson_obj(ρᵗʳⁿ, tᵗʳⁿ,  Hˢʸᵐᵇ, Jˢʸᵐᵇ[1: op_num] )
        
    end # of files (initial states) loop

    return(objₑₓ)
end

g_objective (generic function with 1 method)

In [3]:

function scaling_poly(p::Polynomial)

    X = transpose(hcat([exponents(t) for t in terms(p)]...))

    # Get the scaling via linear regression
    scaling = X \ log.(abs.(coefficients(p)))

    exp.(abs.(scaling)) # 
end

function local_rand_min(p)

    pd = p / maximum(abs.(coefficients(p)))

    # find variable scaling
    scale = scaling_poly(pd)

    # scale the polynomial
    p_scaled = subs(pd, variables(pd) => scale .* variables(pd))

    num_iterations = 100

    # Initialize the best minimizer and the minimum value
    best_minimizer = nothing
    best_min_value = Inf

    num_of_variables = length(variables(pd))

    for _ in 1:num_iterations
        # Generate a random initial condition
        initial_point = rand(num_of_variables).*250

        # Run local optimization
        result = Optim.optimize(p_scaled, initial_point, BFGS())
        #println(Optim.minimum(result))

        # Update the best minimizer if a better one is found
        if Optim.minimum(result) < best_min_value
            
            best_minimizer = Optim.minimizer(result)
            best_min_value = Optim.minimum(result)
            
        end

    end

    best_minimizer = abs.(best_minimizer) # to make gamma positive

    minimizer_scaled = scale .* best_minimizer

    solution = variables(p_scaled) => minimizer_scaled

end

local_rand_min (generic function with 1 method)

In [4]:
data_dir = ""
println(data_dir)

models_dir = "../MODELS/"
#tests_dir = "../TESTS/"
tests_dir = ""

#dodeca_files = ["State_D"*string(n) for n=1:20];

basis_files = ["State_B"*string(n) for n=1:4];

train_files = basis_files 
test_files = ["State_B"*string(n) for n=5:6];

all_files = vcat(train_files, test_files)

6-element Vector{String}:
 "State_B1"
 "State_B2"
 "State_B3"
 "State_B4"
 "State_B5"
 "State_B6"

In [37]:
#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]
γ = [ "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

op_num = 3

tests_data_file_name = "POP_LARGE_GAMMAS_LME_THERM_"*string(op_num)*"_random_trn4_tst2_"*date_and_time_string * ".h5"

for γᵢ in γ

    println("γ =  "*γᵢ)

    poly = g_objective(γᵢ, op_num)
    
    sol = local_rand_min(poly)

    Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol)

    omega = subs(w, sol)

    if op_num >= 1 relaxation = subs(r^2, sol) end
    if op_num >= 3 de_phasing = subs(ϕ^2, sol) end
    #if op_num >= 6 exitation = subs(e^2, sol) end 
    #if op_num >= 4 de_polarisation = subs(p^2, sol) end
    omega = subs(w, sol)

    Jˢⁱᵈ = [DenseOperator(basis, j) for j in subs.(Jˢʸᵐᵇ[1:op_num], sol)]

    J₁ˢⁱᵈ = subs(J₁ˢʸᵐᵇ, sol)
    if op_num >= 2 J₂ˢⁱᵈ = subs(J₂ˢʸᵐᵇ, sol) end
    if op_num >= 3 J₃ˢⁱᵈ = subs(J₃ˢʸᵐᵇ, sol) end

    if op_num >= 4 J₄ˢⁱᵈ = subs(J₄ˢʸᵐᵇ, sol) end
    if op_num >= 5 J₅ˢⁱᵈ = subs(J₅ˢʸᵐᵇ, sol) end
    #J₆ˢⁱᵈ = subs(J₆ˢʸᵐᵇ, sol)


    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ) # create coupling group   

        γ_group["omega"] = convert(Float64, omega)
        γ_group["gamma_relaxation"] = convert(Float64, relaxation)
        #if op_num >= 2 γ_group["gamma_de_phasing"] = convert(Float64,  de_phasing) end
        #if op_num >= 6 γ_group["gamma_exitation"] = convert(Float64,  exitation) end
        if op_num >= 5 γ_group["gamma_de_polarisation"] = convert(Float64, de_polarisation) end
        
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)
        γ_group["J1"] = convert.(ComplexF64, J₁ˢⁱᵈ)
        if op_num >= 2 γ_group["J2"] = convert.(ComplexF64, J₂ˢⁱᵈ) end
        if op_num >= 3 γ_group["J3"] = convert.(ComplexF64, J₃ˢⁱᵈ) end
        if op_num >= 4 γ_group["J4"] = convert.(ComplexF64, J₄ˢⁱᵈ) end
        if op_num >= 5 γ_group["J5"] = convert.(ComplexF64, J₅ˢⁱᵈ) end
        #γ_group["J6"] = convert.(ComplexF64, J₆ˢⁱᵈ)
        
    end

    for df in all_files # loop over initial states
        
        print(df*" ")

        start_time = time()

        #ρₛ, tₛ = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)
        ρₛ, tₛ = LiPoSID.get_rho_series(data_dir*df*"_data.h5", γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        bᵗˢᵗ = LiPoSID.bloch(ρₛ)

        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        tˢⁱᵈ, ρˢⁱᵈ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈ), Jˢⁱᵈ)
        bˢⁱᵈ = LiPoSID.bloch([ρₜ.data for ρₜ in ρˢⁱᵈ])

        Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]                    

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group

            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, Fᴸᴹᴱₑₓ)
            init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
            init_state_group["bloch_sid"] = convert.(Float64, bˢⁱᵈ)
            
        end
                    
        println(minimum(Fᴸᴹᴱₑₓ))
    
    end

    println()
end

println(tests_data_file_name)

γ =  25.133
State_B1 0.9918617575180748
State_B2 0.9915526600583944
State_B3 0.9801465253200439
State_B4 0.9922757082573757
State_B5 0.9794842809213808
State_B6 0.9916270300749337

γ =  79.477
State_B1 0.9987771450006473
State_B2 0.9976171358212328
State_B3 0.9988962375435824
State_B4 0.9973274550109454
State_B5 0.9988897521987654
State_B6 0.9974998027254207

γ =  251.33
State_B1 0.9961693010664818
State_B2 0.9951337561450544
State_B3 0.9956110107718781
State_B4 0.9957115990302993
State_B5 0.9951364536696945
State_B6 0.995678191530871

POP_LARGE_GAMMAS_LME_THERM_3_random_trn4_tst2_2024-Mar-08_at_14-48.h5


γ =  25.133
State_B1 0.9919244297723554

State_B2 0.9914978157292939
State_B3 0.9826153182812789

State_B4 0.9935386208478425
State_B5 0.9777502041370713

State_B6 0.9904549176975618

γ =  79.477
State_B1 
0.9986593706603554
State_B2 0.9976013272504065
State_B3 
0.9963918598611823
State_B4 0.9952178139267989

State_B5 0.995713602747512
State_B6 0.9961809633691978


γ =  251.33
State_B1 0.9961723365287288
State_B2 0.9951298453723714
State_B3 0.9965010590630909

State_B4 0.9957075545398337
State_B5 0.9956001928876012
State_B6 
0.9956533331863862

POP_LARGE_GAMMAS_LME_GEN_1_random_trn4_tst2_2024-Mar-07_at_18-20.h5